In [56]:
import folium
import folium.features
import folium.plugins
from src.constants import WGS84
import geopandas as gpd
import datetime

from src.constants import USER_PATH, PAISAGENSLIDAR_PATH, EBALIDAR_PATH, WGS84, DATA_PATH
from src.utils.os import latest_matching_file

## Load data

In [49]:
paisagens_ground_data_path = latest_matching_file(PAISAGENSLIDAR_PATH / "paisagenslidar_biomasses")
paisagens_ground = gpd.read_file(paisagens_ground_data_path)
paisagens_ground["Survey"] = paisagens_ground["chm_name"]
paisagens_ground = paisagens_ground[~paisagens_ground.geometry.isnull()]
paisagens_ground.head(2)

,location,plotID,sizeplot,chm_name,plot,area_sh_m2,crs_orig,TCHmean,below_c10,over_c10,NApixel,cover10,AGB_plot,ACD,Area [ha],geometry,Survey
0,ANA_A01,P01,50x50,ANA_A01_2018_LAS__02202,plot,2498.649427,+proj=utm +zone=21 +south +datum=WGS84 +units=...,19.456559,237,2277,0,0.905728,349.597758,167.806924,0.250,"POLYGON ((-55.00280 -3.37093, -55.00282 -3.370...",ANA_A01_2018_LAS__02202
1,ANA_A01,P02,50x50,ANA_A01_2018_LAS__02202,plot,2446.014447,+proj=utm +zone=21 +south +datum=WGS84 +units=...,27.960579,50,2404,0,0.979625,440.774902,211.571953,0.245,"POLYGON ((-55.00273 -3.36963, -55.00274 -3.369...",ANA_A01_2018_LAS__02202


In [50]:
paisagens_als_data_path = latest_matching_file(PAISAGENSLIDAR_PATH / "paisagenslidar_file_metadata")
paisagens_als = gpd.read_file(paisagens_als_data_path)
paisagens_als["Area [ha]"] = paisagens_als["area (ha)"]
paisagens_als["Survey"] = paisagens_als["survey"]
paisagens_als.head(2)

,filename,survey,path,crs,las_version,point_format,compressed,creation_date,generating_software,point_count,...,mission,year,region,plot_code,file_size (MB),area (ha),points (Mio.),geometry,Area [ha],Survey
0,CUI_A01_10-01.laz,CUI_A01_2008_laz,/gws/nopw/j04/forecol/data/Paisagenslidar/Miss...,EPSG:32720,1.0,1,True,None,TerraScan,28152592,...,Mission_2008,2008,CUI,A01,115.097,215.97,28.153,"POLYGON ((-60.20210 -2.60807, -60.20213 -2.594...",215.97,CUI_A01_2008_laz
1,CUI_A01_10-02.laz,CUI_A01_2008_laz,/gws/nopw/j04/forecol/data/Paisagenslidar/Miss...,EPSG:32720,1.0,1,True,None,TerraScan,20194370,...,Mission_2008,2008,CUI,A01,82.129,192.19,20.194,"POLYGON ((-60.20155 -2.61980, -60.20158 -2.608...",192.19,CUI_A01_2008_laz


In [51]:
eba_als_data_path = latest_matching_file(EBALIDAR_PATH / "eba_file_metadata")
eba_als = gpd.read_file(eba_als_data_path)
eba_als["Survey"] = eba_als["reduced_filename"]
eba_als["Area [ha]"] = round(eba_als["geometry"].area / 1e4, 3)
eba_als = eba_als.to_crs(WGS84)
eba_als.head(2)

,transect,field_data,hyperspect,obs,campaign,criteria,datafile,elev_maxim,elev_mean,elev_mode,...,canopy_rel,elev_sqrt_,elev_curt_,profile_ar,random,reduced_filename,path,geometry,Survey,Area [ha]
0,0005,None,None,None,1,PRODES+TERRACLASS,NP_T-0005_dn_g_n_ch1_5.laz,62.72,15.2655,15.3311,...,0.602625,17.4348,19.175,38.3784,yes,NP_T-0005.laz,/gws/nopw/j04/forecol/data/EBA_lidar/clouds1/N...,"POLYGON ((-62.33047 3.06104, -62.33042 3.06086...",NP_T-0005.laz,779.474
1,0007,None,None,None,1,PRODES+TERRACLASS,NP_T-0007_dn_g_n_ch1_5.laz,53.81,15.5291,16.7798,...,0.327225,17.4912,18.913,43.9071,yes,NP_T-0007.laz,None,"MULTIPOLYGON (((-62.75839 -0.19807, -62.75832 ...",NP_T-0007.laz,808.751


## Display data

In [52]:
general_cols = ["Survey", "Area [ha]"]
relevant_cols = {
    "paisagens_ground": general_cols + ["location", "plotID", "sizeplot", "ACD", "AGB_plot", "TCHmean", "cover10"],
    "paisagens_als": general_cols + ["points (Mio.)", "year", "region", "plot_code", "filename"],
    "eba_als": general_cols + ["datafile", 'transect', 'field_data', 'obs', 'campaign', "criteria"],
}

eba_available = eba_als[~eba_als.path.isnull()]
eba_unavailable = eba_als[eba_als.path.isnull()]

paisagens_ground_display = paisagens_ground[relevant_cols["paisagens_ground"] + ["geometry"]]
paisagens_als_display = paisagens_als[relevant_cols["paisagens_als"] + ["geometry"]]
eba_als_available_display = eba_available[relevant_cols["eba_als"] + ["geometry"]]
eba_als_unavailable_display = eba_unavailable[relevant_cols["eba_als"] + ["geometry"]]

In [ ]:
# Create map and add layers
world_map = folium.Map(location=(-4.5, -59),
                       control_scale=True,
                       zoom_start=8, 
                       tiles="OpenStreetMap")

# Add minimap
folium.plugins.MiniMap(zoom_level_fixed=2).add_to(world_map)

# Add ESRI satellite imagery
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    overlay = False,
    control = True,
    show=True,
   ).add_to(world_map)

# Add Google Satellite imagery
folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = False,
        control = True
    ).add_to(world_map)

folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True
    ).add_to(world_map)

# Add Google sa
folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite Hybrid',
        overlay = False,
        control = True
    ).add_to(world_map)

# Add latitude longitude 
folium.LatLngPopup().add_to(world_map)

# Add measure control
world_map.add_child(folium.plugins.MeasureControl())

# Add ROI
GROUND_STYLE = {'fillColor': '#db3de6', 'color': '#da3ce6'}
folium.GeoJson(data=paisagens_ground_display.__geo_interface__, 
               name=f"Paisagenslidar ground plots",
               tooltip=folium.features.GeoJsonTooltip(fields=relevant_cols["paisagens_ground"]),
               style_function= lambda x: GROUND_STYLE
              ).add_to(world_map)

# Add ROI
PAISAGENS_STYLE = {'fillColor': '#2a74ac', 'color': '#2a74ac'}
folium.GeoJson(data=paisagens_als_display.__geo_interface__, 
               name=f"Paisagenslidar ALS Transects",
               tooltip=folium.features.GeoJsonTooltip(fields=relevant_cols["paisagens_als"]),
               style_function= lambda x: PAISAGENS_STYLE
              ).add_to(world_map)

# Add ROI (EBA unavailable)
EBA_ROI_UNAVAILABLE_STYLE = {'fillColor': '#ff9999', 'color': '#990000'}
folium.GeoJson(data=eba_als_unavailable_display.__geo_interface__, 
               name="EBA ALS Transects (unavailable)",
               tooltip=folium.features.GeoJsonTooltip(fields=relevant_cols["eba_als"]),
               style_function= lambda x: EBA_ROI_UNAVAILABLE_STYLE
              ).add_to(world_map)

# Add ROI (EBA available)
EBA_ROI_AVAILABLE_STYLE = {'fillColor': '#60BF20', 'color': '#40BF40'}
folium.GeoJson(data=eba_als_available_display.__geo_interface__, 
               name="EBA ALS Transects (available)",
               tooltip=folium.features.GeoJsonTooltip(fields=relevant_cols["eba_als"]),
               style_function= lambda x: EBA_ROI_AVAILABLE_STYLE
              ).add_to(world_map)

# Add map controls
folium.LayerControl().add_to(world_map)

# Add fullscreen
world_map.add_child(folium.plugins.Fullscreen())


world_map.save(f"data_geolocation_map_v{datetime.date.today()}.html")
#world_map.save(DATA_PATH / f"data_geolocation_map_v{datetime.date.today()}.html")
#world_map